In [1]:
from cartoframes.auth import set_default_credentials

set_default_credentials(
    username='your_username',
    api_key='your_api_key'
)

1. Read and understand the data

In [2]:
import pandas
from cartoframes import CartoDataFrame

df = pandas.read_csv('../files/starbucks_brooklyn.csv')
df.head()

,name,address,revenue
0,Franklin Ave & Eastern Pkwy,"341 Eastern Pkwy,Brooklyn, NY 11238",1321040.772
1,607 Brighton Beach Ave,"607 Brighton Beach Avenue,Brooklyn, NY 11235",1268080.418
2,65th St & 18th Ave,"6423 18th Avenue,Brooklyn, NY 11204",1248133.699
3,Bay Ridge Pkwy & 3rd Ave,"7419 3rd Avenue,Brooklyn, NY 11209",1185702.676
4,Caesar's Bay Shopping Center,"8973 Bay Parkway,Brooklyn, NY 11214",1148427.411


2. Before using data services: check the quota

In [3]:
from cartoframes.data.services import Geocoding

gc = Geocoding()
cdf, metadata = gc.geocode(df, street='address', city={'value': 'New York'}, country={'value': 'USA'}, dry_run=True)

Debug: creating table "table_6bc8180ed9"
Success! Data uploaded correctly


In [4]:
metadata

{'total_rows': 10,
 'required_quota': 10,
 'previously_geocoded': 0,
 'previously_failed': 0,
 'records_with_geometry': 0}

Geocode but storing the results using the `cached` option, and that the dataset created is `private` by default

In [5]:
cdf, metadata = gc.geocode(df, street='address', city={'value': 'New York'}, country={'value': 'USA'}, cached='starbucks_cache')

Debug: creating table "starbucks_cache"
Success! Data uploaded correctly
Success! Data geocoded correctly


Compare previous metadata with current metadata

In [6]:
metadata

{'total_rows': 10,
 'required_quota': 10,
 'previously_geocoded': 0,
 'previously_failed': 0,
 'records_with_geometry': 0,
 'final_records_with_geometry': 10,
 'geocoded_increment': 10,
 'successfully_geocoded': 10,
 'failed_geocodings': 0}

Check `gc_status_rel`, `carto_geocode_hash` and `geometry` new columns

In [7]:
cdf.head()

,name,address,revenue,gc_status_rel,carto_geocode_hash,geometry
1,Franklin Ave & Eastern Pkwy,"341 Eastern Pkwy,Brooklyn, NY 11238",1321040.772,0.97,c834a8e289e5bce280775a9bf1f833f1,POINT (-73.95901 40.67109)
2,607 Brighton Beach Ave,"607 Brighton Beach Avenue,Brooklyn, NY 11235",1268080.418,0.99,7d39a3fff93efd9034da88aa9ad2da79,POINT (-73.96122 40.57796)
3,65th St & 18th Ave,"6423 18th Avenue,Brooklyn, NY 11204",1248133.699,0.98,1a2312049ddea753ba42bf77f5ccf718,POINT (-73.98976 40.61912)
4,Bay Ridge Pkwy & 3rd Ave,"7419 3rd Avenue,Brooklyn, NY 11209",1185702.676,0.98,827ab4dcc2d49d5fd830749597976d4a,POINT (-74.02744 40.63152)
5,Caesar's Bay Shopping Center,"8973 Bay Parkway,Brooklyn, NY 11214",1148427.411,0.98,119a38c7b51195cd4153fc81605a8495,POINT (-74.00098 40.59321)


In [8]:
from cartoframes.viz.helpers import color_continuous_layer
from cartoframes.viz import Popup

color_continuous_layer(
    cdf,
    'gc_status_rel',
    title='Geocoding Precision',
    popup=Popup({
        'hover': [{
                'title': 'Address',
                'value': '$address'
            }, {
                'title': 'Precision',
                'value': '$gc_status_rel'
            }]
    })
)

Check available quota

In [9]:
gc.available_quota()

4995667

In [13]:
from cartoframes.data.services import Isolines

iso_service = Isolines()

iso_cdf, metadata = iso_service.isochrones(cdf, [100, 200, 300], mode='car', dry_run=True)

In [11]:
print('available {0}, required {1}'.format(iso_service.available_quota(), metadata.get('required_quota')))

available 111797, required 30


In [41]:
iso_cdf, metadata = iso_service.isochrones(cdf, [100, 200, 300], mode='car')

Debug: creating table "table_357f550006"
Success! Data uploaded correctly
Debug: table "table_357f550006" removed
Success! Isolines correctly created


In [42]:
iso_cdf.head()

,data_range,geometry
1,100,"MULTIPOLYGON (((-73.95902 40.67109, -73.95902 ..."
2,200,"MULTIPOLYGON (((-73.95939 40.67757, -73.95976 ..."
3,300,"MULTIPOLYGON (((-73.95935 40.68038, -73.96128 ..."
4,100,"MULTIPOLYGON (((-73.96158 40.57788, -73.96158 ..."
5,200,"MULTIPOLYGON (((-73.96215 40.58260, -73.96227 ..."


In [51]:
from cartoframes.viz.helpers import color_category_layer

iso_cdf['data_range_cat'] = iso_cdf.apply(lambda row: '{}'.format(row['data_range']), axis=1)
color_category_layer(iso_cdf, 'data_range_cat', opacity='0.2')

Explain `exclusive` parameter, compare data and visualization results

In [44]:
iso_exclusive_cdf, metadata = iso_service.isochrones(cdf, [100, 200, 300], mode='car', exclusive=True)

Debug: creating table "table_c406a8c7dd"
Success! Data uploaded correctly
Debug: table "table_c406a8c7dd" removed
Success! Isolines correctly created


In [45]:
iso_exclusive_cdf.head()

,data_range,lower_data_range,geometry,range_label
1,100,0,"MULTIPOLYGON (((-73.95902 40.67109, -73.95902 ...",2 min.
2,200,100,"POLYGON ((-73.96318 40.67202, -73.96799 40.673...",3 min.
3,300,200,"POLYGON ((-73.95935 40.68039, -73.95848 40.681...",5 min.
4,100,0,"MULTIPOLYGON (((-73.96158 40.57788, -73.96158 ...",2 min.
5,200,100,"MULTIPOLYGON (((-73.96109 40.57523, -73.96133 ...",3 min.


In [46]:
from cartoframes.viz.helpers import isolines_layer

isolines_layer(iso_exclusive_cdf)

Explain other settings available, using isodistances this time:

In [96]:
isodistances_cdf, metadata = iso_service.isodistances(
    cdf,
    [10000, 15000, 20000],
    mode='car',
    exclusive=True,
    is_destination=True,
    mode_traffic='enabled',
    resolution=16.0,
    quality=1,
    dry_run=True
)

In [97]:
metadata.get('required_quota')

30

In [129]:
isodistances_cdf, metadata = iso_service.isodistances(
    cdf,
    [10000, 15000, 20000],
    mode='car',
    exclusive=True,
    is_destination=True,
    mode_traffic='enabled',
    resolution=16.0,
    quality=1
)

Debug: creating table "table_b377756792"
Success! Data uploaded correctly
Debug: table "table_b377756792" removed
Success! Isolines correctly created


In [130]:
isodistances_cdf.head()

,data_range,lower_data_range,geometry,range_label
1,10000,0,"MULTIPOLYGON (((-73.95912 40.67751, -73.96056 ...",167 min.
2,15000,10000,"POLYGON ((-73.95910 40.68184, -73.95865 40.682...",250 min.
3,20000,15000,"POLYGON ((-73.96219 40.66960, -73.96324 40.669...",333 min.
4,10000,0,"MULTIPOLYGON (((-73.96219 40.58278, -73.96226 ...",167 min.
5,15000,10000,"MULTIPOLYGON (((-73.96673 40.57498, -73.96669 ...",250 min.


Multipolygon Empty:


In [131]:
isodistances_cdf = isodistances_cdf[isodistances_cdf['geometry'].map(lambda geometry: not geometry.is_empty)]

In [132]:
isodistances_cdf.head()

,data_range,lower_data_range,geometry,range_label
1,10000,0,"MULTIPOLYGON (((-73.95912 40.67751, -73.96056 ...",167 min.
2,15000,10000,"POLYGON ((-73.95910 40.68184, -73.95865 40.682...",250 min.
3,20000,15000,"POLYGON ((-73.96219 40.66960, -73.96324 40.669...",333 min.
4,10000,0,"MULTIPOLYGON (((-73.96219 40.58278, -73.96226 ...",167 min.
5,15000,10000,"MULTIPOLYGON (((-73.96673 40.57498, -73.96669 ...",250 min.


In [133]:
Layer(isodistances_cdf)

In [134]:
isolines_layer(isodistances_cdf, widget=True)

All together

In [137]:
from cartoframes.viz import Map

Map([
    isolines_layer(isodistances_cdf),
    color_continuous_layer(
        cdf,
        'gc_status_rel',
        title='Geocoding Precision',
        popup=Popup({
            'hover': [{
                    'title': 'Address',
                    'value': '$address'
                }, {
                    'title': 'Precision',
                    'value': '$gc_status_rel'
                }]
        })
    )
])